<a href="https://colab.research.google.com/github/lapythie/ml/blob/master/HW04_ML_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from sklearn.cluster import AffinityPropagation, \
AgglomerativeClustering, DBSCAN, KMeans, MiniBatchKMeans, \
Birch, MeanShift, SpectralClustering

from sklearn.metrics import adjusted_rand_score, \
adjusted_mutual_info_score, silhouette_score

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD, NMF
from sklearn import metrics
from sklearn.datasets import make_blobs

In [ ]:
pd.set_option('display.max_rows', 500)

In [ ]:
def print_metrics(cluster):
    labels = cluster.labels_
    n_clusters = len(set(labels))
    print('CLUSTERS:', '***', len(set(cluster.labels_)), '***')

    try:
        print('Silhouette Coefficient: %0.3f' 
            % metrics.silhouette_score(X, labels))
        print("Homogeneity: %0.3f" % metrics.homogeneity_score(y, labels)) # проверяет, что в кластере объекты одного класса
        print("Completeness: %0.3f" % metrics.completeness_score(y, labels)) # проверяет, что объекты класса только в одном кластере
        print("V-measure: %0.3f" % metrics.v_measure_score(y, labels)) # превращает обе метрики в одну 

        print("Adjusted Rand Index: %0.3f"
            % metrics.adjusted_rand_score(y, labels))
        print("Adjusted Mutual Information: %0.3f"
            % metrics.adjusted_mutual_info_score(y, labels))

    except ValueError as e:
        print(e)

In [ ]:
def print_labels(cluster, sample):
    sample['cluster'] = cluster.labels_
    for i in set(cluster.labels_):
        print(sample[sample.cluster==i]['category_name'].unique())

**Домашнее задание**


## *1. На нескольких алгоритмах кластеризации, умеющих работать с sparse матрицами, проверьте, что работает лучше Count_Vectorizer или TfidfVectorizer (попробуйте выжать максимум из каждого - попробуйте нграммы, символьные нграммы, разные значения max_features и min_df) (3 балла)*

Буду смотреть на v-меру.

In [ ]:
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/homework_ml_m1/HW04_clustering/data.csv')


In [ ]:
# data = pd.read_csv('data.csv')
data = data[['item_id', 'category_name', 'title']]
data.head(5)

,item_id,category_name,title
0,793dd3d5f0f0,Детская одежда и обувь,Платье праздничное для девочки
1,f8c965d9b668,Мебель и интерьер,Кровать двухспальная
2,2f153c00d561,Телефоны,Samsung S3 mini gt-i8190
3,7fc5eea56915,"Одежда, обувь, аксессуары","Красивое платье на одно плечо, в идеале размер м"
4,b01000d29ea3,Детская одежда и обувь,Стильная панама


##AffinityPropagation

In [ ]:
sample = data.sample(frac=0.01, random_state=1)

In [ ]:
print('Категорий:', sample['category_name'].nunique())

Категорий: 47


In [ ]:
sample.shape

(2542, 3)

In [ ]:
cv = CountVectorizer(max_features=800, max_df=0.4, min_df=2)
X = cv.fit_transform(sample['title'])
y = sample['category_name']

In [ ]:
X.shape

(2542, 800)

In [ ]:
%%time
for n2 in [1, 2, 3]:

    cv = CountVectorizer(max_features=800, max_df=0.4, min_df=2, 
                            ngram_range=(1, n2))
    
    X = cv.fit_transform(sample['title'])
    y = sample['category_name']

    cluster = AffinityPropagation(damping=0.7, preference=-2,
                                max_iter=600, verbose=2, )
    cluster.fit(X)
    print_metrics(cluster)

Converged after 311 iterations.
CLUSTERS: *** 656 ***
Silhouette Coefficient: 0.370
Homogeneity: 0.650
Completeness: 0.386
V-measure: 0.485
Adjusted Rand Index: -0.003
Adjusted Mutual Information: 0.223
Converged after 292 iterations.
CLUSTERS: *** 643 ***
Silhouette Coefficient: 0.455
Homogeneity: 0.617
Completeness: 0.377
V-measure: 0.468
Adjusted Rand Index: -0.010
Adjusted Mutual Information: 0.202
Converged after 321 iterations.
CLUSTERS: *** 606 ***
Silhouette Coefficient: 0.478
Homogeneity: 0.607
Completeness: 0.377
V-measure: 0.465
Adjusted Rand Index: -0.012
Adjusted Mutual Information: 0.205
CPU times: user 1min 55s, sys: 42.2 ms, total: 1min 55s
Wall time: 1min 55s


In [ ]:
for n2 in [1, 2, 3]:
    # кажется что сработали лучше униграмы для обоих векторайзеров
    cv = TfidfVectorizer(max_features=500, max_df=0.7, min_df=2,
                         ngram_range=(1, n2))
    X = cv.fit_transform(sample['title'])
    y = sample['category_name']
    cluster = AffinityPropagation(damping=0.7, preference=-2,
                                max_iter=600, verbose=2)
    cluster.fit(X)
    print_metrics(cluster)

Converged after 204 iterations.
CLUSTERS: *** 303 ***
Silhouette Coefficient: 0.443
Homogeneity: 0.577
Completeness: 0.401
V-measure: 0.473
Adjusted Rand Index: -0.009
Adjusted Mutual Information: 0.283
Did not converge
CLUSTERS: *** 1 ***
Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)


/usr/local/lib/python3.6/dist-packages/sklearn/cluster/_affinity_propagation.py:226: ConvergenceWarning: Affinity propagation did not converge, this model will not have any cluster centers.
  "will not have any cluster centers.", ConvergenceWarning)


Did not converge
CLUSTERS: *** 1 ***
Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)


/usr/local/lib/python3.6/dist-packages/sklearn/cluster/_affinity_propagation.py:226: ConvergenceWarning: Affinity propagation did not converge, this model will not have any cluster centers.
  "will not have any cluster centers.", ConvergenceWarning)


In [ ]:
%%time
cluster = AffinityPropagation(damping=0.7, preference=-2,
                             max_iter=600, verbose=2, )
cluster.fit(X)

Converged after 204 iterations.
CPU times: user 25.1 s, sys: 12.8 ms, total: 25.1 s
Wall time: 25.1 s


In [ ]:
print_metrics(cluster)

CLUSTERS: *** 303 ***
Silhouette Coefficient: 0.443
Homogeneity: 0.577
Completeness: 0.401
V-measure: 0.473
Adjusted Rand Index: -0.009
Adjusted Mutual Information: 0.283


##Kmeans

In [ ]:
for an in ['word', 'char', 'char_wb']:
    cv = CountVectorizer(max_features=800, max_df=0.4, min_df=2, 
                         analyzer=an)
    X = cv.fit_transform(sample['title'])
    y = sample['category_name']

    cluster = KMeans(n_clusters=47, random_state=1,
                    verbose=0, n_init=100)
    cluster.fit(X)
    print_metrics(cluster)
    print('\n')

CLUSTERS: *** 47 ***
Silhouette Coefficient: 0.133
Homogeneity: 0.259
Completeness: 0.381
V-measure: 0.308
Adjusted Rand Index: -0.008
Adjusted Mutual Information: 0.235


CLUSTERS: *** 47 ***
Silhouette Coefficient: 0.051
Homogeneity: 0.309
Completeness: 0.275
V-measure: 0.291
Adjusted Rand Index: 0.067
Adjusted Mutual Information: 0.211


CLUSTERS: *** 47 ***
Silhouette Coefficient: 0.051
Homogeneity: 0.309
Completeness: 0.275
V-measure: 0.291
Adjusted Rand Index: 0.067
Adjusted Mutual Information: 0.211




In [ ]:
for feat in np.arange(100,700,100):
    cv = TfidfVectorizer(max_features=feat, max_df=0.7, min_df=2,
                         analyzer='char_wb')
    X = cv.fit_transform(sample['title'])
    y = sample['category_name']

    cluster = KMeans(n_clusters=47, random_state=1,
                    verbose=0, n_init=100)
    cluster.fit(X)
    print_metrics(cluster)
    print('\n')

CLUSTERS: *** 47 ***
Silhouette Coefficient: 0.084
Homogeneity: 0.355
Completeness: 0.290
V-measure: 0.319
Adjusted Rand Index: 0.077
Adjusted Mutual Information: 0.236


CLUSTERS: *** 47 ***
Silhouette Coefficient: 0.084
Homogeneity: 0.355
Completeness: 0.290
V-measure: 0.319
Adjusted Rand Index: 0.077
Adjusted Mutual Information: 0.236


CLUSTERS: *** 47 ***
Silhouette Coefficient: 0.084
Homogeneity: 0.355
Completeness: 0.290
V-measure: 0.319
Adjusted Rand Index: 0.077
Adjusted Mutual Information: 0.236


CLUSTERS: *** 47 ***
Silhouette Coefficient: 0.084
Homogeneity: 0.355
Completeness: 0.290
V-measure: 0.319
Adjusted Rand Index: 0.077
Adjusted Mutual Information: 0.236


CLUSTERS: *** 47 ***
Silhouette Coefficient: 0.084
Homogeneity: 0.355
Completeness: 0.290
V-measure: 0.319
Adjusted Rand Index: 0.077
Adjusted Mutual Information: 0.236


CLUSTERS: *** 47 ***
Silhouette Coefficient: 0.084
Homogeneity: 0.355
Completeness: 0.290
V-measure: 0.319
Adjusted Rand Index: 0.077
Adjusted Mu

##MiniBatchKMeans

In [ ]:
%%time
cv = CountVectorizer(max_features=800, max_df=0.4, min_df=2)
X = cv.fit_transform(sample['title'])
y = sample['category_name']

cluster = MiniBatchKMeans(n_clusters=1000, random_state=1,
                          init_size=5000, max_no_improvement=50,
                          reassignment_ratio=0.3)

cluster.fit(X)
print_metrics(cluster)

CLUSTERS: *** 959 ***
Silhouette Coefficient: 0.431
Homogeneity: 0.718
Completeness: 0.379
V-measure: 0.496
Adjusted Rand Index: 0.002
Adjusted Mutual Information: 0.174
CPU times: user 3.95 s, sys: 8.99 ms, total: 3.96 s
Wall time: 3.97 s


In [ ]:
%%time
cv = TfidfVectorizer(max_features=500, max_df=0.7, min_df=2)
X = cv.fit_transform(data.sample(frac=0.1, random_state=1)['title'])
y = data.sample(frac=0.1, random_state=1)['category_name']

cluster = MiniBatchKMeans(n_clusters=1000, random_state=1,
                          init_size=5000, max_no_improvement=50,
                          reassignment_ratio=0.3)

cluster.fit(X)
print_metrics(cluster)

CLUSTERS: *** 911 ***
Silhouette Coefficient: 0.639
Homogeneity: 0.589
Completeness: 0.327
V-measure: 0.420
Adjusted Rand Index: -0.008
Adjusted Mutual Information: 0.342
CPU times: user 13.5 s, sys: 59 ms, total: 13.5 s
Wall time: 13.5 s


##Birch

In [ ]:
cv = CountVectorizer(max_features=800, max_df=0.4, min_df=2)
X = cv.fit_transform(sample['title'])
y = sample['category_name']

for th in np.arange(0.2, 0.5, 0.05):
    cluster = Birch(threshold=th, branching_factor=50, n_clusters=47, 
                    compute_labels=True, copy=True)

    cluster.fit(X)
    print_metrics(cluster)

CLUSTERS: *** 47 ***
Silhouette Coefficient: 0.145
Homogeneity: 0.327
Completeness: 0.402
V-measure: 0.361
Adjusted Rand Index: 0.006
Adjusted Mutual Information: 0.288
CLUSTERS: *** 47 ***
Silhouette Coefficient: 0.139
Homogeneity: 0.334
Completeness: 0.399
V-measure: 0.364
Adjusted Rand Index: 0.018
Adjusted Mutual Information: 0.291
CLUSTERS: *** 47 ***
Silhouette Coefficient: 0.132
Homogeneity: 0.334
Completeness: 0.413
V-measure: 0.370
Adjusted Rand Index: 0.019
Adjusted Mutual Information: 0.297
CLUSTERS: *** 47 ***
Silhouette Coefficient: 0.140
Homogeneity: 0.329
Completeness: 0.381
V-measure: 0.353
Adjusted Rand Index: 0.007
Adjusted Mutual Information: 0.279
CLUSTERS: *** 47 ***
Silhouette Coefficient: 0.144
Homogeneity: 0.333
Completeness: 0.399
V-measure: 0.363
Adjusted Rand Index: 0.008
Adjusted Mutual Information: 0.290
CLUSTERS: *** 47 ***
Silhouette Coefficient: 0.144
Homogeneity: 0.335
Completeness: 0.406
V-measure: 0.367
Adjusted Rand Index: 0.008
Adjusted Mutual Infor

In [ ]:
cv = TfidfVectorizer(max_features=500, max_df=0.7, min_df=2)
X = cv.fit_transform(sample['title'])
y = sample['category_name']

for fac in np.arange(20, 60, 10):
    cluster = Birch(threshold=0.3, branching_factor=fac, n_clusters=47, 
                    compute_labels=True, copy=True)

    cluster.fit(X)
    print_metrics(cluster)

CLUSTERS: *** 47 ***
Silhouette Coefficient: 0.173
Homogeneity: 0.347
Completeness: 0.388
V-measure: 0.366
Adjusted Rand Index: 0.027
Adjusted Mutual Information: 0.294
CLUSTERS: *** 47 ***
Silhouette Coefficient: 0.158
Homogeneity: 0.368
Completeness: 0.393
V-measure: 0.380
Adjusted Rand Index: 0.010
Adjusted Mutual Information: 0.308
CLUSTERS: *** 47 ***
Silhouette Coefficient: 0.172
Homogeneity: 0.365
Completeness: 0.394
V-measure: 0.379
Adjusted Rand Index: 0.019
Adjusted Mutual Information: 0.307
CLUSTERS: *** 47 ***
Silhouette Coefficient: 0.167
Homogeneity: 0.328
Completeness: 0.372
V-measure: 0.349
Adjusted Rand Index: 0.001
Adjusted Mutual Information: 0.274


In [ ]:
cv = CountVectorizer(max_features=800, max_df=0.4, min_df=2)
X = cv.fit_transform(sample['title'])
y = sample['category_name']

cluster = Birch(threshold=0.3, branching_factor=40, n_clusters=47, 
                compute_labels=True, copy=True)

cluster.fit(X)
print_metrics(cluster)

CLUSTERS: *** 47 ***
Silhouette Coefficient: 0.136
Homogeneity: 0.336
Completeness: 0.409
V-measure: 0.369
Adjusted Rand Index: 0.009
Adjusted Mutual Information: 0.296


In [ ]:
cv = TfidfVectorizer(max_features=500, max_df=0.7, min_df=2)
X = cv.fit_transform(sample['title'])
y = sample['category_name']

cluster = Birch(threshold=0.3, branching_factor=40, n_clusters=47, 
                compute_labels=True, copy=True)

cluster.fit(X)
print_metrics(cluster)

CLUSTERS: *** 47 ***
Silhouette Coefficient: 0.172
Homogeneity: 0.365
Completeness: 0.394
V-measure: 0.379
Adjusted Rand Index: 0.019
Adjusted Mutual Information: 0.307


In [ ]:
sample = data.sample(frac=0.01, random_state=1)

# *2.На нескольких алгоритмах кластеризации проверьте, какое матричное разложение (TruncatedSVD или NMF) работает лучше для кластеризации. (3 балла)*

## DBSCAN

In [ ]:
for n in ['l1', 'l2']:
    # тут ещё немного поподбираем параметры векторизатора
    cv = TfidfVectorizer(max_features=500, max_df=0.7, min_df=2, norm=n)

    svd = TruncatedSVD(50)
    X = cv.fit_transform(sample['title'])
    X = svd.fit_transform(X)

    y = sample['category_name']

    cluster = DBSCAN(min_samples=10, eps=0.6, n_jobs=-1)
    cluster.fit(X)
    print_metrics(cluster)
    print('\n')

CLUSTERS: *** 2 ***
Silhouette Coefficient: 0.516
Homogeneity: 0.004
Completeness: 0.249
V-measure: 0.007
Adjusted Rand Index: 0.000
Adjusted Mutual Information: 0.003


CLUSTERS: *** 2 ***
Silhouette Coefficient: 0.372
Homogeneity: 0.009
Completeness: 0.330
V-measure: 0.017
Adjusted Rand Index: -0.001
Adjusted Mutual Information: 0.012




In [ ]:
for ep in np.arange(0.25, 0.4, 0.05):
    cv = TfidfVectorizer(max_features=500, max_df=0.7, min_df=2, norm='l2')

    svd = TruncatedSVD(50)
    X = cv.fit_transform(sample['title'])
    X = svd.fit_transform(X)

    y = sample['category_name']

    cluster = DBSCAN(min_samples=10, eps=ep, n_jobs=-1)
    cluster.fit(X)
    print_metrics(cluster)
    print('\n')

CLUSTERS: *** 40 ***
Silhouette Coefficient: 0.479
Homogeneity: 0.266
Completeness: 0.393
V-measure: 0.317
Adjusted Rand Index: 0.021
Adjusted Mutual Information: 0.251


CLUSTERS: *** 34 ***
Silhouette Coefficient: 0.455
Homogeneity: 0.232
Completeness: 0.398
V-measure: 0.293
Adjusted Rand Index: 0.002
Adjusted Mutual Information: 0.233


CLUSTERS: *** 38 ***
Silhouette Coefficient: 0.421
Homogeneity: 0.178
Completeness: 0.324
V-measure: 0.230
Adjusted Rand Index: -0.028
Adjusted Mutual Information: 0.155


CLUSTERS: *** 28 ***
Silhouette Coefficient: 0.429
Homogeneity: 0.162
Completeness: 0.344
V-measure: 0.220
Adjusted Rand Index: -0.030
Adjusted Mutual Information: 0.159




In [ ]:
for ms in np.arange(1, 5, 1):
    cv = TfidfVectorizer(max_features=500, max_df=0.7, min_df=2, norm='l2')

    svd = TruncatedSVD(50)
    X = cv.fit_transform(sample['title'])
    X = svd.fit_transform(X)

    y = sample['category_name']

    cluster = DBSCAN(min_samples=ms, eps=0.25, n_jobs=-1)
    cluster.fit(X)
    print_metrics(cluster)
    print('\n')

CLUSTERS: *** 259 ***
Silhouette Coefficient: 0.592
Homogeneity: 0.379
Completeness: 0.379
V-measure: 0.379
Adjusted Rand Index: -0.014
Adjusted Mutual Information: 0.217


CLUSTERS: *** 140 ***
Silhouette Coefficient: 0.601
Homogeneity: 0.359
Completeness: 0.383
V-measure: 0.371
Adjusted Rand Index: -0.012
Adjusted Mutual Information: 0.242


CLUSTERS: *** 98 ***
Silhouette Coefficient: 0.588
Homogeneity: 0.336
Completeness: 0.385
V-measure: 0.359
Adjusted Rand Index: -0.008
Adjusted Mutual Information: 0.248


CLUSTERS: *** 86 ***
Silhouette Coefficient: 0.596
Homogeneity: 0.332
Completeness: 0.385
V-measure: 0.356
Adjusted Rand Index: -0.006
Adjusted Mutual Information: 0.252




In [ ]:
cv = TfidfVectorizer(max_features=500, max_df=0.7, min_df=2, norm='l2')

svd = TruncatedSVD(50)
X = cv.fit_transform(sample['title'])
X = svd.fit_transform(X)

y = sample['category_name']

cluster = DBSCAN(min_samples=1, eps=0.25, n_jobs=-1)
cluster.fit(X)
print_metrics(cluster)

CLUSTERS: *** 257 ***
Silhouette Coefficient: 0.618
Homogeneity: 0.389
Completeness: 0.386
V-measure: 0.387
Adjusted Rand Index: -0.015
Adjusted Mutual Information: 0.227


In [ ]:
cv = TfidfVectorizer(max_features=500, max_df=0.7, min_df=2, norm='l2')

nmf = NMF(50)
X = cv.fit_transform(sample['title'])
X = nmf.fit_transform(X)

y = sample['category_name']

cluster = DBSCAN(min_samples=1, eps=0.25, n_jobs=-1)
cluster.fit(X)
print_metrics(cluster)

CLUSTERS: *** 23 ***
Silhouette Coefficient: 0.341
Homogeneity: 0.015
Completeness: 0.281
V-measure: 0.029
Adjusted Rand Index: -0.004
Adjusted Mutual Information: 0.002


In [ ]:
X = cv.fit_transform(sample['title'])
nmf = NMF(50, random_state=01
X = nmf.fit_transform(X_cv)

Для NMF снова нужно подбирать параметры алгоритма кластеризации. Гридсерч для бедных:

In [ ]:
for ep in np.arange(0.25, 0.4, 0.05):
    for ms in np.arange(1, 5, 1):
        cv = TfidfVectorizer(max_features=500, max_df=0.7, min_df=2, norm='l2')

        nmf = NMF(50)
        X = cv.fit_transform(sample['title'])
        X = nmf.fit_transform(X)
        y = sample['category_name']

        cluster = DBSCAN(min_samples=ms, eps=ep, n_jobs=-1)
        cluster.fit(X)
        print_metrics(cluster)
        print('\n')

CLUSTERS: *** 24 ***
Silhouette Coefficient: 0.322
Homogeneity: 0.015
Completeness: 0.257
V-measure: 0.028
Adjusted Rand Index: -0.004
Adjusted Mutual Information: -0.000


CLUSTERS: *** 18 ***
Silhouette Coefficient: 0.368
Homogeneity: 0.026
Completeness: 0.287
V-measure: 0.048
Adjusted Rand Index: -0.006
Adjusted Mutual Information: 0.014


CLUSTERS: *** 11 ***
Silhouette Coefficient: 0.413
Homogeneity: 0.020
Completeness: 0.278
V-measure: 0.037
Adjusted Rand Index: -0.005
Adjusted Mutual Information: 0.012


CLUSTERS: *** 8 ***
Silhouette Coefficient: 0.405
Homogeneity: 0.015
Completeness: 0.241
V-measure: 0.028
Adjusted Rand Index: -0.005
Adjusted Mutual Information: 0.009


CLUSTERS: *** 5 ***
Silhouette Coefficient: 0.451
Homogeneity: 0.005
Completeness: 0.306
V-measure: 0.009
Adjusted Rand Index: -0.002
Adjusted Mutual Information: 0.003


CLUSTERS: *** 5 ***
Silhouette Coefficient: 0.459
Homogeneity: 0.008
Completeness: 0.326
V-measure: 0.016
Adjusted Rand Index: -0.001
Adjuste

In [ ]:
for s in ['cd', 'mu']:
        cv = TfidfVectorizer(max_features=500, max_df=0.7, min_df=2, norm='l2')

        nmf = NMF(50, random_state=1, shuffle=True,
                  solver=s)
        X = cv.fit_transform(sample['title'])
        X = nmf.fit_transform(X)
        y = sample['category_name']

        cluster = DBSCAN(min_samples=2, eps=0.01, n_jobs=-1)
        cluster.fit(X)
        print_metrics(cluster)
        print('\n')

CLUSTERS: *** 243 ***
Silhouette Coefficient: 0.381
Homogeneity: 0.362
Completeness: 0.345
V-measure: 0.353
Adjusted Rand Index: 0.012
Adjusted Mutual Information: 0.176


CLUSTERS: *** 250 ***
Silhouette Coefficient: 0.421
Homogeneity: 0.380
Completeness: 0.350
V-measure: 0.364
Adjusted Rand Index: 0.012
Adjusted Mutual Information: 0.186




In [ ]:
cv = TfidfVectorizer(max_features=500, max_df=0.7, min_df=2, norm='l2')

nmf = NMF(50, random_state=1, shuffle=True,
            solver='mu', alpha=0.5)
X = cv.fit_transform(sample['title'])
X = nmf.fit_transform(X)
y = sample['category_name']

cluster = DBSCAN(min_samples=2, eps=0.01, n_jobs=-1)
cluster.fit(X)
print_metrics(cluster)
print('\n')

CLUSTERS: *** 248 ***
Silhouette Coefficient: 0.417
Homogeneity: 0.376
Completeness: 0.349
V-measure: 0.362
Adjusted Rand Index: 0.012
Adjusted Mutual Information: 0.184




In [ ]:
cv = TfidfVectorizer(max_features=500, max_df=0.7, min_df=2, norm='l2')

svd = TruncatedSVD(50)
X = cv.fit_transform(sample['title'])
X = svd.fit_transform(X)

y = sample['category_name']

cluster = SpectralClustering(n_clusters=47)
cluster.fit(X)
print_metrics(cluster)

CLUSTERS: *** 47 ***
Silhouette Coefficient: 0.568
Homogeneity: 0.291
Completeness: 0.412
V-measure: 0.341
Adjusted Rand Index: -0.007
Adjusted Mutual Information: 0.262


In [ ]:
cv = TfidfVectorizer(max_features=500, max_df=0.7, min_df=2, norm='l2')

nmf = NMF(50, random_state=1, shuffle=True,
            solver='mu', alpha=0.5)
X = cv.fit_transform(sample['title'])
X = nmf.fit_transform(X)
y = sample['category_name']
cluster = SpectralClustering(n_clusters=47)
cluster.fit(X)
print_metrics(cluster)

CLUSTERS: *** 47 ***
Silhouette Coefficient: 0.654
Homogeneity: 0.337
Completeness: 0.401
V-measure: 0.366
Adjusted Rand Index: -0.000
Adjusted Mutual Information: 0.291


## MeanShift


In [ ]:
cv = TfidfVectorizer(max_features=500, max_df=0.7, min_df=2, norm='l2')

svd = TruncatedSVD(50)
X = cv.fit_transform(sample['title'])
X = svd.fit_transform(X)

y = sample['category_name']

cluster = MeanShift(cluster_all=False, bandwidth=0.8, n_jobs=-1)
cluster.fit(X)
print_metrics(cluster)

CLUSTERS: *** 4 ***
Silhouette Coefficient: 0.370
Homogeneity: 0.026
Completeness: 0.135
V-measure: 0.044
Adjusted Rand Index: -0.015
Adjusted Mutual Information: 0.032


In [ ]:
for bd in np.arange(0.1,0.6,0.1):
    cv = TfidfVectorizer(max_features=500, max_df=0.7, min_df=2, norm='l2')

    nmf = NMF(50, random_state=1, shuffle=True,
                solver='mu', alpha=0.5)
    X = cv.fit_transform(sample['title'])
    X = nmf.fit_transform(X)

    y = sample['category_name']

    cluster = MeanShift(cluster_all=False, bandwidth=bd, n_jobs=-1)
    cluster.fit(X)
    print_metrics(cluster)

CLUSTERS: *** 256 ***
Silhouette Coefficient: 0.742
Homogeneity: 0.403
Completeness: 0.373
V-measure: 0.387
Adjusted Rand Index: -0.008
Adjusted Mutual Information: 0.225
CLUSTERS: *** 117 ***
Silhouette Coefficient: 0.643
Homogeneity: 0.346
Completeness: 0.390
V-measure: 0.366
Adjusted Rand Index: -0.005
Adjusted Mutual Information: 0.262
CLUSTERS: *** 41 ***
Silhouette Coefficient: 0.514
Homogeneity: 0.147
Completeness: 0.282
V-measure: 0.193
Adjusted Rand Index: -0.032
Adjusted Mutual Information: 0.120
CLUSTERS: *** 9 ***
Silhouette Coefficient: 0.444
Homogeneity: 0.041
Completeness: 0.202
V-measure: 0.068
Adjusted Rand Index: -0.020
Adjusted Mutual Information: 0.044
CLUSTERS: *** 2 ***
Silhouette Coefficient: 0.471
Homogeneity: 0.005
Completeness: 0.139
V-measure: 0.010
Adjusted Rand Index: -0.003
Adjusted Mutual Information: 0.005


Не с каждым алгоритмом SVD работает лучше.

# *3.С помощью алгоритмов, умеющих выделять выбросы, попробуйте найти необычные объявления (необычные - это такие, которые непонятно к какой категории можно вообще отнести, что-то с ошибками или вообще какая-то дичь). В этом задании можно использовать любую векторизацию. (4 балла)*

In [ ]:

cv = TfidfVectorizer(max_features=500, max_df=0.7, min_df=2, norm='l2')

nmf = NMF(50, random_state=1, shuffle=True,
            solver='mu', alpha=0.5)
X = cv.fit_transform(sample['title'])
X = nmf.fit_transform(X)

y = sample['category_name']

cluster = MeanShift(cluster_all=False, bandwidth=0.1, n_jobs=-1)
cluster.fit(X)
print_metrics(cluster)

CLUSTERS: *** 256 ***
Silhouette Coefficient: 0.742
Homogeneity: 0.403
Completeness: 0.373
V-measure: 0.387
Adjusted Rand Index: -0.008
Adjusted Mutual Information: 0.225


In [ ]:
sample['cluster'] = cluster.labels_

In [ ]:
len(sample[sample.cluster==-1]

65

In [ ]:
cv = TfidfVectorizer(max_features=500, max_df=0.7, min_df=2, norm='l2')

svd = TruncatedSVD(50)
X = cv.fit_transform(sample['title'])
X = svd.fit_transform(X)

y = sample['category_name']

cluster = MeanShift(cluster_all=False, bandwidth=0.1, n_jobs=-1)
cluster.fit(X)
print_metrics(cluster)

CLUSTERS: *** 503 ***
Silhouette Coefficient: 0.610
Homogeneity: 0.493
Completeness: 0.367
V-measure: 0.421
Adjusted Rand Index: -0.013
Adjusted Mutual Information: 0.190


In [ ]:
sample['cluster'] = cluster.labels_
sample[sample.cluster==-1]

,item_id,category_name,title,cluster
101395,1ac31610b2ad,Мебель и интерьер,Кушетка от мебельной мастерской Арт Мебель,-1
157057,09fb729ea8ad,Товары для компьютера,DDR2 512MB для пк в ассортименте,-1
212157,a657aa41a81c,Планшеты и электронные книги,Планшет Irbis TZ-42 на запчасти,-1
2419,160643c4cf2a,"Игры, приставки и программы","X-BOX 360, LT 3.0, 120 GB",-1
139619,1f2334ba150c,Автомобили,"Audi Q7, 2012",-1
153512,f218d8b0b562,Мотоциклы и мототехника,Honda dio 35 150cc,-1
145982,98415361d1cb,Мебель и интерьер,"Коврик для ванной 1,25х0,92 м",-1
163646,d8c269379cc7,Товары для животных,Отдам отсадник для рыб,-1
157239,05f99f10d24c,"Одежда, обувь, аксессуары",Демисезонная слингокуртка,-1
217042,96dc2c284567,"Одежда, обувь, аксессуары",Плащ куртка женская,-1


Непонятно, почему это стало выбросами. По-моему, обычные названия. Возможно, в них слишком много редких токенов. Типа александрита, рублей и СССР.

In [ ]:

from tqdm import tqdm
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
cluster = MiniBatchKMeans(n_clusters=1000, random_state=1,
                          init_size=5000, max_no_improvement=50,
                          reassignment_ratio=0.3)

cluster.fit(X)
print_metrics(cluster)

CLUSTERS: *** 938 ***
Silhouette Coefficient: 0.588
Homogeneity: 0.684
Completeness: 0.370
V-measure: 0.480
Adjusted Rand Index: -0.007
Adjusted Mutual Information: 0.150


In [ ]:
scores = []
# tqdm._instances.clear()
for i in tqdm(range(5, 3000, 50)):
    cluster = MiniBatchKMeans(n_clusters=i, random_state=1, 
                              init_size=i*3, max_no_improvement=50,
                              reassignment_ratio=0.3)
    cluster.fit(X)
    # силуэты считать на всех данных считать долго
    # можно взять внутреннюю метрику кминса
    # она большая, поэтому можно взять корень
    scores.append(np.sqrt(cluster.inertia_))

 85%|████████▌ | 51/60 [04:46<01:34, 10.48s/it]

ValueError: ignored

Не получилось! Видно, накосячила с параметрами и векторизация была плохой.